# Tests output quality
**Purpose:** (Human) evaluation of our solution outputs, on the following kinds of aspects: 
- Is the classification of the exercise correct?
- Do the corrections seem appropriate? Do the scores seem fair? 
- Do the Pro tips seem useful? How long does processing take (on average)? 
- What errors appear?

## Setup

In [1]:
# Import packages - Note "google-cloud-aiplatform" should be installed and upgraded
import os
import json
import csv
import time
from IPython.display import HTML
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
from google.cloud import storage

# Available Gemini models
gemini_latest = "gemini-1.5-pro-latest" # Latest version, only for testing or prototyping
gemini_latest_stable = "gemini-1.5-pro" # Latest stable version, ready for prod
gemini_flash_latest_stable = "gemini-1.5-flash"

# Low-bar safety filters
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

## Helper functions

In [2]:
def load_video_part(gcs_uri):
    """Loads a video from a URI and creates a corresponding Part object.
    Args:
        uri (str): The GCS URI of the video.
    Returns:
        Part: The loaded video as a Part object.
    """
    file_extension = gcs_uri.split('.')[-1]  # Get the file extension
    mime_type = {
        'mov': "video/quicktime",
        'mp4': "video/mp4"
    }.get(file_extension)

    if mime_type is None:
        raise ValueError("Unsupported video format. Only .mov and .mp4 are currently supported.")

    return Part.from_uri(mime_type=mime_type, uri=gcs_uri)


def exercise_text_info(exercise):
    """Fetches and returns information from the exercises.json file."""
    with open('./exercises.json', 'r') as file:  # Assuming JSON is in the same directory
        data = json.load(file)

    exercise_data = data.get(exercise)

    if exercise_data is None:
        return f"Exercise {exercise} not found in exercises.json"

    text_output = ""

    # Iterate over main keys (e.g., "form", "tempo")
    for main_key, main_value in exercise_data.items():
        text_output += f"\n{main_key}:\n"
        if isinstance(main_value, dict): # CHECK IF IT IS A DICT FIRST
            # Iterate over nested items (e.g., "Steps", "Form" under "form")
            for inner_key, inner_value in main_value.items():
                if isinstance(inner_value, str):
                    formatted_value = inner_value.replace('\\n', '\n')
                    text_output += f"  {inner_key}: {formatted_value}\n"
        else:  # Handle the case where main_value is not a dictionary (e.g., a string)
            text_output += f"  {main_value}\n" # Add string value directly to output

    return text_output

## Call 1 (original): LLM to classify exercise

In [3]:
# Array with all exercises indexes (for classification)
exercises_index_list = """["band-assisted_bench_press", "bar_dip", "bench_press", "bench_press_against_band", "board_press", "cable_chest_press", "close-grip_bench_press", "close-grip_feet-up_bench_press", "decline_bench_press", "decline_push-up", "dumbbell_chest_fly", "dumbbell_chest_press", "dumbbell_decline_chest_press", "dumbbell_floor_press", "dumbbell_pullover", "feet-up_bench_press", "floor_press", "incline_bench_press", "incline_dumbbell_press", "incline_push-up", "kettlebell_floor_press", "kneeling_incline_push-up", "kneeling_push-up", "machine_chest_fly", "machine_chest_press", "pec_deck", "pin_bench_press", "push-up", "push-up_against_wall", "push-ups_with_feet_in_rings", "resistance_band_chest_fly", "smith_machine_bench_press", "smith_machine_incline_bench_press", "standing_cable_chest_fly", "standing_resistance_band_chest_fly", "band_external_shoulder_rotation", "band_internal_shoulder_rotation", "band_pull-apart", "barbell_front_raise", "barbell_rear_delt_row", "barbell_upright_row", "behind_the_neck_press", "cable_lateral_raise", "cable_rear_delt_row", "dumbbell_front_raise", "dumbbell_horizontal_internal_shoulder_rotation", "dumbbell_horizontal_external_shoulder_rotation", "dumbbell_lateral_raise", "dumbbell_rear_delt_row", "dumbbell_shoulder_press", "face_pull", "front_hold", "lying_dumbbell_external_shoulder_rotation", "lying_dumbbell_internal_shoulder_rotation", "machine_lateral_raise", "machine_shoulder_press", "monkey_row", "overhead_press", "plate_front_raise", "power_jerk", "push_press", "reverse_cable_flyes", "reverse_dumbbell_flyes", "reverse_machine_fly", "seated_dumbbell_shoulder_press", "seated_barbell_overhead_press", "seated_smith_machine_shoulder_press", "snatch_grip_behind_the_neck_press", "squat_jerk", "split_jerk", "barbell_curl", "barbell_preacher_curl", "bodyweight_curl", "cable_curl_with_bar", "cable_curl_with_rope", "concentration_curl", "dumbbell_curl", "dumbbell_preacher_curl", "hammer_curl", "incline_dumbbell_curl", "machine_bicep_curl", "spider_curl", "barbell_standing_triceps_extension", "barbell_lying_triceps_extension", "bench_dip", "close-grip_push-up", "dumbbell_lying_triceps_extension", "dumbbell_standing_triceps_extension", "overhead_cable_triceps_extension", "tricep_bodyweight_extension", "tricep_pushdown_with_bar", "tricep_pushdown_with_rope", "air_squat", "barbell_hack_squat", "barbell_lunge", "barbell_walking_lunge", "belt_squat", "body_weight_lunge", "bodyweight_leg_curl", "box_squat", "bulgarian_split_squat", "chair_squat", "dumbbell_lunge", "dumbbell_squat", "front_squat", "goblet_squat", "hack_squat_machine", "half_air_squat", "hip_adduction_machine", "jumping_lunge", "landmine_hack_squat", "landmine_squat", "leg_curl_on_ball", "leg_extension", "leg_press", "lying_leg_curl", "nordic_hamstring_eccentric", "pause_squat", "reverse_barbell_lunge", "romanian_deadlift", "safety_bar_squat", "seated_leg_curl", "shallow_body_weight_lunge", "side_lunges_(bodyweight)", "smith_machine_squat", "barbell_squat", "step_up", "zercher_squat", "assisted_chin-up", "assisted_pull-up", "back_extension", "banded_muscle-up", "barbell_row", "barbell_shrug", "block_clean", "block_snatch", "cable_close_grip_seated_row", "cable_wide_grip_seated_row", "chin-up", "clean", "clean_and_jerk", "deadlift", "deficit_deadlift", "dumbbell_deadlift", "dumbbell_row", "dumbbell_shrug", "floor_back_extension", "good_morning", "hang_clean", "hang_power_clean", "hang_power_snatch", "hang_snatch", "inverted_row", "inverted_row_with_underhand_grip", "jefferson_curl", "jumping_muscle-up", "kettlebell_swing", "lat_pulldown_with_pronated_grip", "lat_pulldown_with_supinated_grip", "muscle-up_(bar)", "muscle-up_(rings)", "one-handed_cable_row", "one-handed_lat_pulldown", "pause_deadlift", "pendlay_row", "power_clean", "power_snatch", "pull-up", "pull-up_with_a_neutral_grip", "rack_pull", "ring_pull-up", "ring_row", "seal_row", "seated_machine_row", "snatch", "snatch_grip_deadlift", "stiff-legged_deadlift", "straight_arm_lat_pulldown", "sumo_deadlift", "t-bar_row", "trap_bar_deadlift_with_high_handles", "trap_bar_deadlift_with_low_handles", "banded_side_kicks", "cable_pull_through", "clamshells", "dumbbell_romanian_deadlift", "dumbbell_frog_pumps", "fire_hydrants", "frog_pumps", "glute_bridge", "hip_abduction_against_band", "hip_abduction_machine", "hip_thrust", "hip_thrust_machine", "hip_thrust_with_band_around_knees", "lateral_walk_with_band", "machine_glute_kickbacks", "one-legged_glute_bridge", "one-legged_hip_thrust", "reverse_hyperextension", "romanian_deadlift", "single_leg_romanian_deadlift", "standing_glute_kickback_in_machine", "step_up", "ball_slams", "cable_crunch", "crunch", "dead_bug", "hanging_knee_raise", "hanging_leg_raise", "hanging_sit-up", "high_to_low_wood_chop_with_band", "horizontal_wood_chop_with_band", "kneeling_ab_wheel_roll-out", "kneeling_plank", "kneeling_side_plank", "lying_leg_raise", "lying_windshield_wiper", "lying_windshield_wiper_with_bent_knees", "machine_crunch", "mountain_climbers", "oblique_crunch", "oblique_sit-up", "plank", "plank_with_leg_lifts", "side_plank", "sit-up", "barbell_standing_calf_raise", "eccentric_heel_drop", "heel_raise", "seated_calf_raise", "standing_calf_raise", "barbell_wrist_curl", "barbell_wrist_curl_behind_the_back", "bar_hang", "dumbbell_wrist_curl", "farmers_walk", "fat_bar_deadlift", "gripper", "one-handed_bar_hang", "plate_pinch", "plate_wrist_curl", "towel_pull-up", "barbell_wrist_extension", "dumbbell_wrist_extension", "rowing_machine", "stationary_bike"]"""

# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
    "top_p": 0.95,
}

# Call definition
def classify_exercise(user_video_uri):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(gemini_latest_stable)
    responses = model.generate_content(
        [
            """
            You're an AI Coach. You need to identify the exercise that the user is performing:
            """,
            load_video_part(user_video_uri),
            """
            and classify it as one of the exercise indexes in this list:
            """,
            exercises_index_list,
            """
            Respond only with the exercise index, followed by a line break:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Classified exercise: {classified_exercise}\n")
    return classified_exercise

## Call 2: Multimodal model to do the magic and retrieve final response

In [21]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.0,
    "top_p": 0.95,
    "response_mime_type":"application/json",
}

def generate(user_video_uri, classified_exercise):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(gemini_latest_stable)
    responses = model.generate_content(
        [f"""
        You're an AI Coach, an AI-based software that gives corrections or suggestions on how a user performs a strength exercise.
        You need to follow a number of steps in order to give scores and potentially suggest improvements to the user.
        Speak directly to the user in the second person, always in a friendly and motivational tone.
        
        1. The user is performing
        """
        , classified_exercise,
        """
         exercise. This is their video:
        """
        , load_video_part(user_video_uri),
        """
        2. Now, this is all the reference information you have, as an expert coach, on that exercise:
        """
        , exercise_text_info(classified_exercise),
        """
        Note on "tempo": Tempo is a way to describe the speed of each phase of a repetition. It's usually written as a four-digit number (e.g., 2010). Here's what each digit represents:
            - 1st digit: Eccentric phase (lowering the weight)
            - 2nd digit: Pause at the bottom of the movement
            - 3rd digit: Concentric phase (lifting the weight)
            - 4th digit: Pause at the top of the movement
        
        
        3. Compare the user exercise video with the reference content based on the former information. You should do two things with that comparison:
        3.1. Scores. Give a score and suggest improvements to the user as precisely as posibble, taking into account that the reference content includes the right way to perform the exercise.
        Score is a % where 0% means that the user is not following the theory at all, and a 100% means that the user is covering that category perfectly.
        - Give "100%" if the user is doing perfect in that category (form, tempo, or range of movement) and you don't find any improvements.
        - Also don't be afraid of giving lower (even really low) scores when needed.
        As part of each score, also suggest all aspects that the user could improve on for each category.
        Finally, give pro tips and and (when applicable) improvements suggestions with all the reference content you have on that exercises.
        
        3.2. Pro tips. Two kinds of "pro tips" you should give to the user in a friendly way:
        
        3.2.1. Detect which reference information can be useful to the user, based on their execution. In an objective while friendly way, let them know things that may be interesting for them.
        These pro tips could be related to potential variations they could do on the form, tempo, range of movement depending on their objectives, or any other aspect that may seem interesting.
        
        3.2.2. (Only if you detect decreases or deviations that the user could having throughout the video, between reps of this same serie - Prioritise this pro tip if so). 
        You could detect changes in the velocity that they are performing (e.g. if they are slowing down each rep), changes in their range of movement, or in their form.
        Objective of this section is trying to spot potential signs of fatigue. While there's research on fatigue's impact on performance, identifying precise visual cues for each individual remains a challenge.
        We do know that fatigue can lead to decreased force production, motor control issues, and an increased risk of injury. Visually, this might manifest as:
        - Tempo: Slowing down of movement, especially in the concentric (lifting) phase.
        - Range of Movement: Decreases in the full range of movement, possibly due to compensatory movements.
        - Form Breakdown: Deviation from proper technique, such as arching the back during squats or leaning forward during rows.
        There's significant individual variation in how fatigue manifests. Factors like training experience, fitness level, and exercise selection play a role. This makes establishing universal visual cues difficult.
        While not foolproof, a potential sign to look out for is with general Observation: Noticeable changes in movement quality, speed, and form compared to earlier repetitions.
        Consequences of Fatigue: Pushing through fatigue can increase the risk of injury, decrease performance, and hinder recovery. It's crucial to recognize the signs and adjust accordingly.
        When to Stop or Adjust: There's no one-size-fits-all answer, but general guidelines include:
        - Form Breakdown: If proper form can't be maintained, stop the set or reduce the weight.
        - Pain: Any sharp or unusual pain warrants immediate cessation of the exercise.
        - Self-Assessment: Listen to your body. If you feel excessively fatigued, stop or modify the workout.
        Alternatives to Continuing:
        - Lower the weight: Use a lighter load that allows for proper form and a full range of movement.
        - Change exercises: Switch to a variation that targets the same muscle groups but places less stress on fatigued areas.
        - Rest: Take a short break to allow for recovery before continuing (e.g. split into more series with less reps).
        If you indentify that a user is having any of the fatigue potential manifestations, ask them to do an exercise of self-Reporting:
        If the user reports feeling fatigued or struggling to maintain proper form throughout th exercise, let them know when to stop or adjust, and alternatives to continuing.
        
        Output should be a valid JSON similar to:
            "Scores": {
                "Form": {
                    "Score": "... (%)",
                    "Improvement Suggestions": "..."
                }
                "Tempo": {
                    "Score": "... (%)",
                    "Improvement Suggestions": "..."
                }
                "Range of Movement": {
                    "Score": "... (%)",
                    "Improvement Suggestions": "..."
                }
            
            "Pro tips": {
                "...:": "...",
                "...": "...",
                "...": "..."
             }
        """
        ],
        
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    generated_output = "".join(response.text.strip() for response in responses)
    print(f"JSON generated for {classified_exercise}\n")
    return generated_output

## Load test videos for model input

For this test, I selected a subset of 14 videos. Each video represents a different exercise.

In [34]:
videos_folder = "gs://gymally-mvp/exercises-videos"

# GCS URIs for the 14 videos to test on
test_videos = [
    f"{videos_folder}/abduction-usr1-20240527-1.mp4",
    f"{videos_folder}/bar_dip_ref_strengthlog.mp4",
    f"{videos_folder}/bench_press_against_band_ref_strengthlog.mp4",
    f"{videos_folder}/bench-press-usr4-20240527-1.mp4",
    f"{videos_folder}/board_bench_press_ref_strengthlog.mp4",
    f"{videos_folder}/bulgarian-squat-usr1-20240617-1.mp4",
    f"{videos_folder}/cable_chest_press_ref_strengthlog.mp4",
    f"{videos_folder}/deadlift_usr1_20240627_2.mp4",
    f"{videos_folder}/hip-thrust-usr1-20240523-2.mp4",
    f"{videos_folder}/kettlebell-deadlift-usr1-20240701-1.mp4",
    f"{videos_folder}/leg-extension-usr2-20240606-1.mp4",
    f"{videos_folder}/leg-press-usr1-20240530-1.mp4",
    f"{videos_folder}/leg-pull-usr1-20240606-1.mp4",
    f"{videos_folder}/unknown-exercise-usr2-20240509-1.mp4"
]

## Call models (Loop)
The following loop goes through the 14 exercises above, runs the complete cycle (both calls), and writes in a CSV file named as `test_name`, including the columns:
- Video processed
- Test (iteration number for that video, 1-3)
- Call 1 output (classified_exercise)
- Call 2 output (generated JSON)
- Total time to process that video (since video is loaded until final result is returned to the user)

In case of any error, the error message gets logged to the CSV too.

In [33]:
def process_videos(videos, test_name="test"):
    file_path = f'{test_name}_output.csv'
    file_exists = os.path.isfile(file_path)
    
    # Load already processed exercises if the file exists
    processed_exercises = set()
    if file_exists:
        with open(file_path, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                processed_exercises.add(row['Call 1 output'])
    
    for video_uri in videos:
        results = []
        for test in range(1, 4):  # Perform 3 tests for each video
            start_time = time.time()  # Start time for processing the video
            try:
                classified_exercise = classify_exercise(video_uri)  # Moved inside the loop
                if classified_exercise in processed_exercises:
                    print(f"Skipping already processed exercise: {classified_exercise}")
                    break  # Skip the rest of the tests for this video
                generated_json = generate(video_uri, classified_exercise)
                end_time = time.time()  # End time for processing the video
                total_time = end_time - start_time  # Calculate total processing time for this iteration
                results.append((video_uri, test, classified_exercise, generated_json, f"{total_time:.2f} seconds"))
            except Exception as e:
                print(f"Error processing {video_uri}: {e}")
                end_time = time.time()  # Ensure end time is captured even on error
                total_time = end_time - start_time
                results.append((video_uri, test, "Error", str(e), f"{total_time:.2f} seconds"))
        
        # Write results to CSV, append if file exists
        with open(file_path, 'a', newline='') as csvfile:
            fieldnames = ['input video URI', 'test', 'Call 1 output', 'Call 2 output', 'Total Processing Time']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if not file_exists:
                writer.writeheader()
                file_exists = True  # Ensure header is not written again
            
            for result in results:
                writer.writerow({
                    'input video URI': result[0], 
                    'test': result[1], 
                    'Call 1 output': result[2], 
                    'Call 2 output': result[3],
                    'Total Processing Time': result[4]
                })
        if results:  # Only print if there were results processed
            print(f"Processed and saved results for {video_uri}")

In [34]:
process_videos(test_videos, 'test_14_exercises_x3')

Classified exercise: machine_abduction_adduction

JSON generated for machine_abduction_adduction

Classified exercise: machine_abduction_adduction

JSON generated for machine_abduction_adduction

Classified exercise: machine_abduction_adduction

JSON generated for machine_abduction_adduction

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise: bar_dip

JSON generated for bar_dip

Classified exercise: bar_dip

JSON generated for bar_dip

Classified exercise: bar_dip

JSON generated for bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise: bench_press_against_band

JSON generated for bench_press_against_band

Classified exercise: bench_press_against_band

JSON generated for bench_press_against_band

Classified exercise: bench_press_against_band

JSON generated for bench_press_against_band

Processed and saved results for gs://gymally-mvp/exercises-videos/

**Conclusion:** After analysing the results, there are several issues we could try to improve (See [Issue tracker detected prior to Alpha](https://docs.google.com/document/d/1K4wP26NfjoNLr1xWofJu8PDTFgNVCIUCgP7U3ob1IpU/edit#heading=h.r8lxxzc6aw7n)).

# Testing potential improvement hypothesis
## Issue 1. Output is not one of the exercises in `exercise_list`, or output in a wrong format.

### Experiment 1.1. Include system instructions + prompt engineering to try to limit response to exercise list (including format)

Let's edit Call 1 configuration to include system instructions:

In [12]:
# Call 1 (updated)

# Array with all exercises indexes (for classification)
exercises_index_list = """["band-assisted_bench_press", "bar_dip", "bench_press", "bench_press_against_band", "board_press", "cable_chest_press", "close-grip_bench_press", "close-grip_feet-up_bench_press", "decline_bench_press", "decline_push-up", "dumbbell_chest_fly", "dumbbell_chest_press", "dumbbell_decline_chest_press", "dumbbell_floor_press", "dumbbell_pullover", "feet-up_bench_press", "floor_press", "incline_bench_press", "incline_dumbbell_press", "incline_push-up", "kettlebell_floor_press", "kneeling_incline_push-up", "kneeling_push-up", "machine_chest_fly", "machine_chest_press", "pec_deck", "pin_bench_press", "push-up", "push-up_against_wall", "push-ups_with_feet_in_rings", "resistance_band_chest_fly", "smith_machine_bench_press", "smith_machine_incline_bench_press", "standing_cable_chest_fly", "standing_resistance_band_chest_fly", "band_external_shoulder_rotation", "band_internal_shoulder_rotation", "band_pull-apart", "barbell_front_raise", "barbell_rear_delt_row", "barbell_upright_row", "behind_the_neck_press", "cable_lateral_raise", "cable_rear_delt_row", "dumbbell_front_raise", "dumbbell_horizontal_internal_shoulder_rotation", "dumbbell_horizontal_external_shoulder_rotation", "dumbbell_lateral_raise", "dumbbell_rear_delt_row", "dumbbell_shoulder_press", "face_pull", "front_hold", "lying_dumbbell_external_shoulder_rotation", "lying_dumbbell_internal_shoulder_rotation", "machine_lateral_raise", "machine_shoulder_press", "monkey_row", "overhead_press", "plate_front_raise", "power_jerk", "push_press", "reverse_cable_flyes", "reverse_dumbbell_flyes", "reverse_machine_fly", "seated_dumbbell_shoulder_press", "seated_barbell_overhead_press", "seated_smith_machine_shoulder_press", "snatch_grip_behind_the_neck_press", "squat_jerk", "split_jerk", "barbell_curl", "barbell_preacher_curl", "bodyweight_curl", "cable_curl_with_bar", "cable_curl_with_rope", "concentration_curl", "dumbbell_curl", "dumbbell_preacher_curl", "hammer_curl", "incline_dumbbell_curl", "machine_bicep_curl", "spider_curl", "barbell_standing_triceps_extension", "barbell_lying_triceps_extension", "bench_dip", "close-grip_push-up", "dumbbell_lying_triceps_extension", "dumbbell_standing_triceps_extension", "overhead_cable_triceps_extension", "tricep_bodyweight_extension", "tricep_pushdown_with_bar", "tricep_pushdown_with_rope", "air_squat", "barbell_hack_squat", "barbell_lunge", "barbell_walking_lunge", "belt_squat", "body_weight_lunge", "bodyweight_leg_curl", "box_squat", "bulgarian_split_squat", "chair_squat", "dumbbell_lunge", "dumbbell_squat", "front_squat", "goblet_squat", "hack_squat_machine", "half_air_squat", "hip_adduction_machine", "jumping_lunge", "landmine_hack_squat", "landmine_squat", "leg_curl_on_ball", "leg_extension", "leg_press", "lying_leg_curl", "nordic_hamstring_eccentric", "pause_squat", "reverse_barbell_lunge", "romanian_deadlift", "safety_bar_squat", "seated_leg_curl", "shallow_body_weight_lunge", "side_lunges_(bodyweight)", "smith_machine_squat", "barbell_squat", "step_up", "zercher_squat", "assisted_chin-up", "assisted_pull-up", "back_extension", "banded_muscle-up", "barbell_row", "barbell_shrug", "block_clean", "block_snatch", "cable_close_grip_seated_row", "cable_wide_grip_seated_row", "chin-up", "clean", "clean_and_jerk", "deadlift", "deficit_deadlift", "dumbbell_deadlift", "dumbbell_row", "dumbbell_shrug", "floor_back_extension", "good_morning", "hang_clean", "hang_power_clean", "hang_power_snatch", "hang_snatch", "inverted_row", "inverted_row_with_underhand_grip", "jefferson_curl", "jumping_muscle-up", "kettlebell_swing", "lat_pulldown_with_pronated_grip", "lat_pulldown_with_supinated_grip", "muscle-up_(bar)", "muscle-up_(rings)", "one-handed_cable_row", "one-handed_lat_pulldown", "pause_deadlift", "pendlay_row", "power_clean", "power_snatch", "pull-up", "pull-up_with_a_neutral_grip", "rack_pull", "ring_pull-up", "ring_row", "seal_row", "seated_machine_row", "snatch", "snatch_grip_deadlift", "stiff-legged_deadlift", "straight_arm_lat_pulldown", "sumo_deadlift", "t-bar_row", "trap_bar_deadlift_with_high_handles", "trap_bar_deadlift_with_low_handles", "banded_side_kicks", "cable_pull_through", "clamshells", "dumbbell_romanian_deadlift", "dumbbell_frog_pumps", "fire_hydrants", "frog_pumps", "glute_bridge", "hip_abduction_against_band", "hip_abduction_machine", "hip_thrust", "hip_thrust_machine", "hip_thrust_with_band_around_knees", "lateral_walk_with_band", "machine_glute_kickbacks", "one-legged_glute_bridge", "one-legged_hip_thrust", "reverse_hyperextension", "romanian_deadlift", "single_leg_romanian_deadlift", "standing_glute_kickback_in_machine", "step_up", "ball_slams", "cable_crunch", "crunch", "dead_bug", "hanging_knee_raise", "hanging_leg_raise", "hanging_sit-up", "high_to_low_wood_chop_with_band", "horizontal_wood_chop_with_band", "kneeling_ab_wheel_roll-out", "kneeling_plank", "kneeling_side_plank", "lying_leg_raise", "lying_windshield_wiper", "lying_windshield_wiper_with_bent_knees", "machine_crunch", "mountain_climbers", "oblique_crunch", "oblique_sit-up", "plank", "plank_with_leg_lifts", "side_plank", "sit-up", "barbell_standing_calf_raise", "eccentric_heel_drop", "heel_raise", "seated_calf_raise", "standing_calf_raise", "barbell_wrist_curl", "barbell_wrist_curl_behind_the_back", "bar_hang", "dumbbell_wrist_curl", "farmers_walk", "fat_bar_deadlift", "gripper", "one-handed_bar_hang", "plate_pinch", "plate_wrist_curl", "towel_pull-up", "barbell_wrist_extension", "dumbbell_wrist_extension", "rowing_machine", "stationary_bike"]"""

# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
    "top_p": 0.95,
}

# Call definition
def classify_exercise(user_video_uri):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the exercise that the user is performing.
            It's crucial to classify the exercise accurately, choosing exclusively from the following predefined exercise indexes:
            """,
            exercises_index_list,
            "Please classify the exercise by selecting ONLY from the above list. Respond with the exact exercise index, followed by a line break."
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Classified exercise: {classified_exercise}\n")
    return classified_exercise

Run testing loop, but just using the Call 1 (only classifying the exercise, not continuing with Call 2 to produce scores and feedback).

In [7]:
def process_videos(videos, test_name="test"):
    file_path = f'{test_name}_output.csv'
    file_exists = os.path.isfile(file_path)
    
    # Load already processed exercises if the file exists
    processed_exercises = set()
    
    for video_uri in videos:
        results = []
        for test in range(1, 4):  # Perform 3 tests for each video
            start_time = time.time()  # Start time for processing the video
            try:
                classified_exercise = classify_exercise(video_uri)  # Moved inside the loop
                if classified_exercise in processed_exercises:
                    print(f"Skipping already processed exercise: {classified_exercise}")
                    break  # Skip the rest of the tests for this video
                end_time = time.time()  # End time for processing the video
                total_time = end_time - start_time  # Calculate total processing time for this iteration
                results.append((video_uri, test, classified_exercise, f"{total_time:.2f} seconds"))
            except Exception as e:
                print(f"Error processing {video_uri}: {e}")
                end_time = time.time()  # Ensure end time is captured even on error
                total_time = end_time - start_time
                results.append((video_uri, test, "Error", str(e), f"{total_time:.2f} seconds"))
        
        # Write results to CSV, append if file exists
        with open(file_path, 'a', newline='') as csvfile:
            fieldnames = ['input video URI', 'test', 'Call 1 output', 'Total Processing Time']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if not file_exists:
                writer.writeheader()
                file_exists = True  # Ensure header is not written again
            
            for result in results:
                writer.writerow({
                    'input video URI': result[0], 
                    'test': result[1], 
                    'Call 1 output': result[2], 
                    'Total Processing Time': result[3]
                })
        if results:  # Only print if there were results processed
            print(f"Processed and saved results for {video_uri}")

In [8]:
process_videos(test_videos, 'test_classification_adhere_to_exercise_list')

Classified exercise: hip_abduction_machine

Classified exercise: hip_abduction_machine

Classified exercise: hip_abduction_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise: bar_dip

Classified exercise: bar_dip

Classified exercise: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise: bench_press_against_band

Classified exercise: band-assisted_bench_press

Classified exercise: bench_press_against_band

Processed and saved results for gs://gymally-mvp/exercises-videos/bench_press_against_band_ref_strengthlog.mp4
Classified exercise: bench_press

Classified exercise: bench_press

Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/bench-press-usr4-20240527-1.mp4
Classified exercise: board_press

Classified exercise: board_press

Error processing gs://gymally-mvp/exercises-videos/board_bench_pres

**Conclusion:** 
- Format and adherance to `exercise_list` increased to 100% with our subset of 14 exercises. 
- Accuracy of classification also improved from 43% to 74%.

## Issue 2. Confused exercises for no evident reason

Model is not able to regognise basic exercises like bulgarian split or deadlift.

### Experiment 2.1. Check if model is confused due to `exercise_list` or different ways to call these exercises, or if it is really not capable to recognise these exercises

In [25]:
videos_folder = "gs://gymally-mvp/exercises-videos"

user_video_1 = f"{videos_folder}/bulgarian-squat-usr1-20240617-1.mp4"
user_video_2 = f"{videos_folder}/deadlift_usr1_20240627_2.mp4"
user_video_3 = f"{videos_folder}/kettlebell-deadlift-usr1-20240701-1.mp4"
user_video_4 = f"{videos_folder}/unknown-exercise-usr2-20240509-1.mp4"
user_video_5 = f"{videos_folder}/leg-extension-usr2-20240606-1.mp4"   

In [36]:
# Call 1 (updated)

# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.0,
    "top_k": 1,
}

# Call definition
def classify_exercise(user_video_uri, model_used):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = model_used,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the exercise that the user is performing.
            It's crucial to classify the exercise accurately. Respond only with the exercise name identified.
            """,
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Classified exercise: {classified_exercise}\n")
    return classified_exercise

### Experiment 2.2. Try the same as in Experiment 1.1., but using Gemini-flash, temp=0 and top_k=1 instead

In [32]:
# Call 1 (updated)

# Array with all exercises indexes (for classification)
exercises_index_list = """["band-assisted_bench_press", "bar_dip", "bench_press", "bench_press_against_band", "board_press", "cable_chest_press", "close-grip_bench_press", "close-grip_feet-up_bench_press", "decline_bench_press", "decline_push-up", "dumbbell_chest_fly", "dumbbell_chest_press", "dumbbell_decline_chest_press", "dumbbell_floor_press", "dumbbell_pullover", "feet-up_bench_press", "floor_press", "incline_bench_press", "incline_dumbbell_press", "incline_push-up", "kettlebell_floor_press", "kneeling_incline_push-up", "kneeling_push-up", "machine_chest_fly", "machine_chest_press", "pec_deck", "pin_bench_press", "push-up", "push-up_against_wall", "push-ups_with_feet_in_rings", "resistance_band_chest_fly", "smith_machine_bench_press", "smith_machine_incline_bench_press", "standing_cable_chest_fly", "standing_resistance_band_chest_fly", "band_external_shoulder_rotation", "band_internal_shoulder_rotation", "band_pull-apart", "barbell_front_raise", "barbell_rear_delt_row", "barbell_upright_row", "behind_the_neck_press", "cable_lateral_raise", "cable_rear_delt_row", "dumbbell_front_raise", "dumbbell_horizontal_internal_shoulder_rotation", "dumbbell_horizontal_external_shoulder_rotation", "dumbbell_lateral_raise", "dumbbell_rear_delt_row", "dumbbell_shoulder_press", "face_pull", "front_hold", "lying_dumbbell_external_shoulder_rotation", "lying_dumbbell_internal_shoulder_rotation", "machine_lateral_raise", "machine_shoulder_press", "monkey_row", "overhead_press", "plate_front_raise", "power_jerk", "push_press", "reverse_cable_flyes", "reverse_dumbbell_flyes", "reverse_machine_fly", "seated_dumbbell_shoulder_press", "seated_barbell_overhead_press", "seated_smith_machine_shoulder_press", "snatch_grip_behind_the_neck_press", "squat_jerk", "split_jerk", "barbell_curl", "barbell_preacher_curl", "bodyweight_curl", "cable_curl_with_bar", "cable_curl_with_rope", "concentration_curl", "dumbbell_curl", "dumbbell_preacher_curl", "hammer_curl", "incline_dumbbell_curl", "machine_bicep_curl", "spider_curl", "barbell_standing_triceps_extension", "barbell_lying_triceps_extension", "bench_dip", "close-grip_push-up", "dumbbell_lying_triceps_extension", "dumbbell_standing_triceps_extension", "overhead_cable_triceps_extension", "tricep_bodyweight_extension", "tricep_pushdown_with_bar", "tricep_pushdown_with_rope", "air_squat", "barbell_hack_squat", "barbell_lunge", "barbell_walking_lunge", "belt_squat", "body_weight_lunge", "bodyweight_leg_curl", "box_squat", "bulgarian_split_squat", "chair_squat", "dumbbell_lunge", "dumbbell_squat", "front_squat", "goblet_squat", "hack_squat_machine", "half_air_squat", "hip_adduction_machine", "jumping_lunge", "landmine_hack_squat", "landmine_squat", "leg_curl_on_ball", "leg_extension", "leg_press", "lying_leg_curl", "nordic_hamstring_eccentric", "pause_squat", "reverse_barbell_lunge", "romanian_deadlift", "safety_bar_squat", "seated_leg_curl", "shallow_body_weight_lunge", "side_lunges_(bodyweight)", "smith_machine_squat", "barbell_squat", "step_up", "zercher_squat", "assisted_chin-up", "assisted_pull-up", "back_extension", "banded_muscle-up", "barbell_row", "barbell_shrug", "block_clean", "block_snatch", "cable_close_grip_seated_row", "cable_wide_grip_seated_row", "chin-up", "clean", "clean_and_jerk", "deadlift", "deficit_deadlift", "dumbbell_deadlift", "dumbbell_row", "dumbbell_shrug", "floor_back_extension", "good_morning", "hang_clean", "hang_power_clean", "hang_power_snatch", "hang_snatch", "inverted_row", "inverted_row_with_underhand_grip", "jefferson_curl", "jumping_muscle-up", "kettlebell_swing", "lat_pulldown_with_pronated_grip", "lat_pulldown_with_supinated_grip", "muscle-up_(bar)", "muscle-up_(rings)", "one-handed_cable_row", "one-handed_lat_pulldown", "pause_deadlift", "pendlay_row", "power_clean", "power_snatch", "pull-up", "pull-up_with_a_neutral_grip", "rack_pull", "ring_pull-up", "ring_row", "seal_row", "seated_machine_row", "snatch", "snatch_grip_deadlift", "stiff-legged_deadlift", "straight_arm_lat_pulldown", "sumo_deadlift", "t-bar_row", "trap_bar_deadlift_with_high_handles", "trap_bar_deadlift_with_low_handles", "banded_side_kicks", "cable_pull_through", "clamshells", "dumbbell_romanian_deadlift", "dumbbell_frog_pumps", "fire_hydrants", "frog_pumps", "glute_bridge", "hip_abduction_against_band", "hip_abduction_machine", "hip_thrust", "hip_thrust_machine", "hip_thrust_with_band_around_knees", "lateral_walk_with_band", "machine_glute_kickbacks", "one-legged_glute_bridge", "one-legged_hip_thrust", "reverse_hyperextension", "romanian_deadlift", "single_leg_romanian_deadlift", "standing_glute_kickback_in_machine", "step_up", "ball_slams", "cable_crunch", "crunch", "dead_bug", "hanging_knee_raise", "hanging_leg_raise", "hanging_sit-up", "high_to_low_wood_chop_with_band", "horizontal_wood_chop_with_band", "kneeling_ab_wheel_roll-out", "kneeling_plank", "kneeling_side_plank", "lying_leg_raise", "lying_windshield_wiper", "lying_windshield_wiper_with_bent_knees", "machine_crunch", "mountain_climbers", "oblique_crunch", "oblique_sit-up", "plank", "plank_with_leg_lifts", "side_plank", "sit-up", "barbell_standing_calf_raise", "eccentric_heel_drop", "heel_raise", "seated_calf_raise", "standing_calf_raise", "barbell_wrist_curl", "barbell_wrist_curl_behind_the_back", "bar_hang", "dumbbell_wrist_curl", "farmers_walk", "fat_bar_deadlift", "gripper", "one-handed_bar_hang", "plate_pinch", "plate_wrist_curl", "towel_pull-up", "barbell_wrist_extension", "dumbbell_wrist_extension", "rowing_machine", "stationary_bike"]"""

# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.0,
    "top_k": 1,
}

# Call definition
def classify_exercise(user_video_uri):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_flash_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the exercise that the user is performing.
            It's crucial to classify the exercise accurately, choosing exclusively from the following predefined exercise indexes:
            """,
            exercises_index_list,
            "Please classify the exercise by selecting ONLY from the above list. Respond with the exact exercise index, followed by a line break."
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Classified exercise: {classified_exercise}\n")
    return classified_exercise

In [8]:
def process_videos(videos, test_name="test"):
    file_path = f'{test_name}_output.csv'
    file_exists = os.path.isfile(file_path)
    
    # Load already processed exercises if the file exists
    processed_exercises = set()
    
    for video_uri in videos:
        results = []
        for test in range(1, 3):  # Perform 2 tests for each video (with temp=0 and top_k=1, responses should be (almost) always the same)
            start_time = time.time()  # Start time for processing the video
            try:
                classified_exercise = classify_exercise(video_uri)  # Moved inside the loop
                if classified_exercise in processed_exercises:
                    print(f"Skipping already processed exercise: {classified_exercise}")
                    break  # Skip the rest of the tests for this video
                end_time = time.time()  # End time for processing the video
                total_time = end_time - start_time  # Calculate total processing time for this iteration
                results.append((video_uri, test, classified_exercise, f"{total_time:.2f} seconds"))
            except Exception as e:
                print(f"Error processing {video_uri}: {e}")
                end_time = time.time()  # Ensure end time is captured even on error
                total_time = end_time - start_time
                results.append((video_uri, test, "Error", str(e), f"{total_time:.2f} seconds"))
        
        # Write results to CSV, append if file exists
        with open(file_path, 'a', newline='') as csvfile:
            fieldnames = ['input video URI', 'test', 'Call 1 output', 'Total Processing Time']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if not file_exists:
                writer.writeheader()
                file_exists = True  # Ensure header is not written again
            
            for result in results:
                writer.writerow({
                    'input video URI': result[0], 
                    'test': result[1], 
                    'Call 1 output': result[2], 
                    'Total Processing Time': result[3]
                })
        if results:  # Only print if there were results processed
            print(f"Processed and saved results for {video_uri}")

In [35]:
process_videos(test_videos, '20240626_test_classification_flash_temp0_topk1')

Classified exercise: hip_abduction_machine

Classified exercise: hip_abduction_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise: bar_dip

Classified exercise: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise: bench_press_against_band

Classified exercise: bench_press_against_band

Processed and saved results for gs://gymally-mvp/exercises-videos/bench_press_against_band_ref_strengthlog.mp4
Classified exercise: bench_press

Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/bench-press-usr4-20240527-1.mp4
Classified exercise: bench_press

Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/board_bench_press_ref_strengthlog.mp4
Classified exercise: dumbbell_romanian_deadlift

Classified exercise: dumbbell_romanian_deadlift

Processed and saved r

**Conclusion:** Gemini Flash seems to identify exercises better when not restricted to an `exercise_list`, but seems to get confused when needing to chose from these 250 exercises.

## Issue 3. Restricting model to `exercise_list` could be producing worse results. 
### Experiment 3.1. One call for classification vs two calls
- **Option 1:** One call only - Call 1 to classify video into an exercise from `exercise_list` (like until now)
- **Option 2:** Two Calls: Call 1 to output an exercise based on video (with freedom of response), Call 2 to match output from Call 1 with `exercise_list`

In [6]:
# Load videos for model input: Elena's + Pablo's

videos_folder = "gs://gymally-mvp/exercises-videos"

# GCS URIs for the 21 videos to test on
test_videos = [
    f"{videos_folder}/abduction-usr1-20240527-1.mp4",
    f"{videos_folder}/bar_dip_ref_strengthlog.mp4",
    f"{videos_folder}/bench_press_against_band_ref_strengthlog.mp4",
    f"{videos_folder}/bench-press-usr4-20240527-1.mp4",
    f"{videos_folder}/board_bench_press_ref_strengthlog.mp4",
    f"{videos_folder}/bulgarian-squat-usr1-20240617-1.mp4",
    f"{videos_folder}/cable_chest_press_ref_strengthlog.mp4",
    f"{videos_folder}/deadlift_usr1_20240627_2.mp4",
    f"{videos_folder}/hip-thrust-usr1-20240523-2.mp4",
    f"{videos_folder}/kettlebell-deadlift-usr1-20240701-1.mp4",
    f"{videos_folder}/leg-extension-usr2-20240606-1.mp4",
    f"{videos_folder}/leg-press-usr1-20240530-1.mp4",
    f"{videos_folder}/leg-pull-usr1-20240606-1.mp4",
    f"{videos_folder}/unknown-exercise-usr2-20240509-1.mp4",
    f"{videos_folder}/IMG_3621.mov",
    f"{videos_folder}/IMG_3622.mov",
    f"{videos_folder}/IMG_3623.mov",
    f"{videos_folder}/IMG_3624.mov",
    f"{videos_folder}/IMG_3625.mov",
    f"{videos_folder}/IMG_3626.mov",
    f"{videos_folder}/IMG_3627.mov",
]

#### Option 1.1 - One call only, Gemini-Pro, temp = 0.1

In [5]:
# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
}

# Call definition
def classify_exercise(user_video_uri):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the exercise that the user is performing.
            It's crucial to classify the exercise accurately, choosing exclusively from the following predefined exercise indexes:
            """,
            exercises_index_list,
            "Please classify the exercise by selecting ONLY from the above list. Respond with the exact exercise index, followed by a line break."
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Classified exercise: {classified_exercise}\n")
    return classified_exercise

In [9]:
process_videos(test_videos, '20240629_test_class_freedom_1call')

Classified exercise: hip_abduction_machine

Classified exercise: hip_abduction_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise: bar_dip

Classified exercise: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise: bench_press_against_band

Classified exercise: bench_press_against_band

Processed and saved results for gs://gymally-mvp/exercises-videos/bench_press_against_band_ref_strengthlog.mp4
Classified exercise: bench_press

Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/bench-press-usr4-20240527-1.mp4
Classified exercise: board_press

Classified exercise: board_press

Processed and saved results for gs://gymally-mvp/exercises-videos/board_bench_press_ref_strengthlog.mp4
Classified exercise: push-up_against_wall

Classified exercise: push-up_against_wall

Processed and saved results for g

### Option 2.1. Two calls, Gemini pro with temp 0.2 (call 1) and temp=0.0, top_k=1 (call 2)

In [10]:
# CALL 1 - Classify exercise with freedom

# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.2,
}

# Call definition
def classify_exercise_free(user_video_uri):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the exercise that the user is performing.
            It's crucial to classify the strength exercise accurately, based on your knowledge on strength exercises.
            """
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise_free = "".join(response.text.strip() for response in responses)
    classified_exercise_free = classified_exercise_free.strip('"')

    print(f"Classified exercise with freedom: {classified_exercise_free}\n")
    return classified_exercise_free


# CALL 2 - Match classified exercise with exercise list
# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.0,
    "top_k": 1,
}

# Call definition
def classify_exercise_list(classified_exercise_free):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to take a string as an input, representing a strength exercises, and match it with one exercise from an input list.
            It's crucial to classively onefy the exercise accurately, choosing exclusi from the following predefined exercise indexes:
            """,
            exercises_index_list,
            "Please classify the exercise by selecting ONLY from the above list. Respond with the exact exercise index identified, followed by a line break."
        ]
    )
    responses = model.generate_content(
        [
            """
            Input exercise string:
            """,
            classified_exercise_free,
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Exercise match in exercise list: {classified_exercise}\n")
    return classified_exercise

In [12]:
# Loop over 21 videos (only once per video) and process them, saving the results to a CSV file

def process_videos(videos, test_name="test"):
    file_path = f'{test_name}.csv'
    file_exists = os.path.isfile(file_path)
    
    for video_uri in videos:
        results = []
        start_time = time.time()  # Start time for processing the video
        try:
            classified_exercise_free = classify_exercise_free(video_uri)  # Moved inside the loop
            classified_exercise = classify_exercise_list(classified_exercise_free)
            end_time = time.time()  # End time for processing the video
            total_time = end_time - start_time  # Calculate total processing time for this iteration
            results.append((video_uri, classified_exercise_free, classified_exercise, f"{total_time:.2f} seconds"))
        except Exception as e:
            print(f"Error processing {video_uri}: {e}")
            end_time = time.time()  # Ensure end time is captured even on error
            total_time = end_time - start_time
            results.append((video_uri, "Error", str(e), f"{total_time:.2f} seconds"))
        
        # Write results to CSV, append if file exists
        with open(file_path, 'a', newline='') as csvfile:
            fieldnames = ['input video URI', 'Call 1 output', 'Call 2 output', 'Total Processing Time']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if not file_exists:
                writer.writeheader()
                file_exists = True  # Ensure header is not written again
            
            for result in results:
                writer.writerow({
                    'input video URI': result[0], 
                    'Call 1 output': result[1], 
                    'Call 2 output': result[2],
                    'Total Processing Time': result[3]
                })
        if results:  # Only print if there were results processed
            print(f"Processed and saved results for {video_uri}")

In [13]:
process_videos(test_videos, '20240629_test_class_2calls')

Classified exercise with freedom: Theuser in the video is performing **Torso Rotation** on a machine specifically designedfor this exercise.

Exercise match in exercise list: rowing_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise with freedom: Theuser in the video is performing **dips**.

Exercise match in exercise list: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise with freedom: Theuser is performing a **Banded Barbell Bench Press**. 

Here's why:

* **Barbell and Bench:** The user is lying ona weight bench and pressing a barbell, which are the defining characteristics of a bench press.
* **Resistance Bands:** The presence of resistance bands attached to thebarbell indicates a variation of the standard bench press. The bands add variable resistance, making the exercise harder at the top of the movement. 

Let me knowif you have any o

#### Experiment 2.2. Same approach, but using Gemini-Flash and with a slight prompt engineering
Prompt engineering purpose is to simplify call 1 answer

In [19]:
# CALL 1 - Classify exercise with freedom

# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.2,
}

# Call definition
def classify_exercise_free(user_video_uri):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_flash_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the exercise that the user is performing.
            It's crucial to identify the strength exercise accurately, based on your knowledge on strength exercises.
            Return the complete name of the identified exercise (if exercise is called in different ways, return both).
            Specify the material being used, e.g. a machine, assisted, with some other material (specify which), against a resistance, with the own body...
            """
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise_free = "".join(response.text.strip() for response in responses)
    classified_exercise_free = classified_exercise_free.strip('"')

    print(f"Classified exercise with freedom: {classified_exercise_free}\n")
    return classified_exercise_free


# CALL 2 - Match classified exercise with exercise list
# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.0,
    "top_k": 1,
}

# Call definition
def classify_exercise_list(classified_exercise_free):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_flash_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to take a string as an input, representing a strength exercises, and match it with one exercise from an input list.
            It's crucial to classify the exercise accurately, choosing exclusively one from the following predefined exercise indexes:
            """,
            exercises_index_list,
            "Please classify the exercise by selecting ONLY from the above list. Respond with the exact exercise index identified, followed by a line break."
        ]
    )
    responses = model.generate_content(
        [
            """
            Input exercise string:
            """,
            classified_exercise_free,
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Exercise match in exercise list: {classified_exercise}\n")
    return classified_exercise

In [20]:
process_videos(test_videos, '20240629_test_class_2calls_flash')

Classified exercise with freedom: Theexercise is **Hip Abduction** performed on a **machine**.

Exercise match in exercise list: hip_abduction_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise with freedom: Theuser is performing **Dips** on a **dip station**.

Exercise match in exercise list: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise with freedom: Theuser is performing a **Bench Press** with a barbell on a bench.

Exercise match in exercise list: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/bench_press_against_band_ref_strengthlog.mp4
Classified exercise with freedom: Theexercise is **Barbell Bench Press**. The user is performing the exercise with abarbell, lying on a bench.

Exercise match in exercise list: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/bench-press-u

#### Experiment 2.3. Same approach, using Gemini-Flash, with more prompt engineering
Prompt engineering purpose is to better interpret the material being used in the video.

In [28]:
# CALL 1 - Classify exercise with freedom

# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
}

# Call definition
def classify_exercise_free(user_video_uri):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_flash_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the exercise that the user is performing.
            It's crucial to identify the strength exercise accurately, based on your knowledge on strength exercises.
            Return the complete name of the identified exercise (if exercise is called in different ways, return both).
            If the user is using any materials apart from their own bodu, pecify the material that the user is leveraging: e.g. are they using a gym machine, or a bench and one barbell, or two barbells, is the exercise being assisted, which other materials is the user leveraging (resistance bands, dumbbels, kettlebells, a board, any other thing...?). Are these materials assisting the exercise of placing resistance?
            """
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise_free = "".join(response.text.strip() for response in responses)
    classified_exercise_free = classified_exercise_free.strip('"')

    print(f"Classified exercise with freedom: {classified_exercise_free}\n")
    return classified_exercise_free


# CALL 2 - Match classified exercise with exercise list
# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.0,
    "top_k": 1,
}

# Call definition
def classify_exercise_list(classified_exercise_free):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_flash_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to take a string as an input, representing a strength exercises, and match it with one exercise from an input list.
            It's crucial to classify the exercise accurately, choosing exclusively one from the following predefined exercise indexes:
            """,
            exercises_index_list,
            "Please classify the exercise by selecting ONLY from the above list. Respond with the exact exercise index identified, followed by a line break."
        ]
    )
    responses = model.generate_content(
        [
            """
            Input exercise string:
            """,
            classified_exercise_free,
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Exercise match in exercise list: {classified_exercise}\n")
    return classified_exercise

In [29]:
process_videos(test_videos, '20240630_test_class_2calls_flash')

Classified exercise with freedom: Theuser is performing a **Hip Abduction Machine** exercise. The user is leveraginga gym machine to assist the exercise and place resistance.

Exercise match in exercise list: hip_abduction_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise with freedom: Theuser is performing **Dips**. The user is leveraging a **Dip Station**to perform the exercise.

Exercise match in exercise list: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise with freedom: Theuser is performing a **Bench Press** with a barbell and a resistance band.The resistance band is assisting the exercise by providing additional resistance.

Exercise match in exercise list: bench_press_against_band

Processed and saved results for gs://gymally-mvp/exercises-videos/bench_press_against_band_ref_strengthlog.mp4
Classified exercise with freedom: Theuse

**Conclusion:** Making it one call seems to be getting better results. Let's try now to improve the prompt with the findings from the 2 calls experiments: Making the model pay attention to the material used seems to work.

### Experiment 3.1. One call only, using Gemini-flash and providing some COT

In [32]:
# Call 1 (updated)

# Model configuration
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.0,
    "top_k": 1,
}

# Call definition
def classify_exercise(user_video_uri):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = gemini_flash_latest_stable,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the strength exercise that the user is performing.
            Think step by step. It's crucial to identify the exercise accurately, based on your knowledge on strength exercises.
            In order to identify the exercise, evaluate aspects like:
            - Which exercise is the user performing? E.g. Is it a deadlift, some kind of squat, some kind of pull-up... any other kind of exercise? 
            - Which material is the user leveraging? E.g. a gym machine, a bench and one barbell, two barbells, resistance bands, dumbbels, kettlebells, a board, any other thing...?
            - Which group muscles does the user seem to be targeting?

            Based on your analysis, it's crucial to classify the exercise accurately, choosing exclusively from the following predefined exercise indexes:
            """,
            exercises_index_list,
            "Please classify the exercise by selecting ONLY from the above list. Respond with the exact exercise index, followed by a line break."
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Classified exercise: {classified_exercise}\n")
    return classified_exercise

In [34]:
# Loop over 21 videos (only once per video) and process them, saving the results to a CSV file

def process_videos(videos, test_name="test"):
    file_path = f'{test_name}.csv'
    file_exists = os.path.isfile(file_path)
    
    for video_uri in videos:
        results = []
        start_time = time.time()  # Start time for processing the video
        try:
            classified_exercise = classify_exercise(video_uri)  # Moved inside the loop
            end_time = time.time()  # End time for processing the video
            total_time = end_time - start_time  # Calculate total processing time for this iteration
            results.append((video_uri, classified_exercise, f"{total_time:.2f} seconds"))
        except Exception as e:
            print(f"Error processing {video_uri}: {e}")
            end_time = time.time()  # Ensure end time is captured even on error
            total_time = end_time - start_time
            results.append((video_uri, "Error", str(e), f"{total_time:.2f} seconds"))
        
        # Write results to CSV, append if file exists
        with open(file_path, 'a', newline='') as csvfile:
            fieldnames = ['input video URI', 'Call 1 (only) output', 'Total Processing Time']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if not file_exists:
                writer.writeheader()
                file_exists = True  # Ensure header is not written again
            
            for result in results:
                writer.writerow({
                    'input video URI': result[0], 
                    'Call 1 (only) output': result[1], 
                    'Total Processing Time': result[2]
                })
        if results:  # Only print if there were results processed
            print(f"Processed and saved results for {video_uri}")

In [35]:
process_videos(test_videos, '20240630_test_class_1call_flash_cot')

Classified exercise: hip_abduction_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise: bench_press_against_band

Processed and saved results for gs://gymally-mvp/exercises-videos/bench_press_against_band_ref_strengthlog.mp4
Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/bench-press-usr4-20240527-1.mp4
Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/board_bench_press_ref_strengthlog.mp4
Classified exercise: Theuser is performing a **dumbbell_lunge**. 
The user isleveraging a pair of dumbbells.
The user seems to be targeting the quadriceps, hamstrings, glutes, and calves.

Processed and saved results for gs://gymally-mvp/exercises-videos/bulgarian-squat-usr1-20240617-1.mp4
Classified exerci

**Conclusion:** This prompt may be over-complex, as 

## Issue 4. Choose best set of model = parameters

Let's try combining:
- Gemini 1.5 Pro, Gemini 1.5 Flash
- Temperature 0.0 (top_k=1), 0.1, 0.2

With our 21 videos and choose the best set for exercise classification.

In [8]:
# Redefine exercises_index_list changing "deadlift" to "barbell_deadlift"
exercises_index_list = """["band-assisted_bench_press", "bar_dip", "bench_press", "bench_press_against_band", "board_press", "cable_chest_press", "close-grip_bench_press", "close-grip_feet-up_bench_press", "decline_bench_press", "decline_push-up", "dumbbell_chest_fly", "dumbbell_chest_press", "dumbbell_decline_chest_press", "dumbbell_floor_press", "dumbbell_pullover", "feet-up_bench_press", "floor_press", "incline_bench_press", "incline_dumbbell_press", "incline_push-up", "kettlebell_floor_press", "kneeling_incline_push-up", "kneeling_push-up", "machine_chest_fly", "machine_chest_press", "pec_deck", "pin_bench_press", "push-up", "push-up_against_wall", "push-ups_with_feet_in_rings", "resistance_band_chest_fly", "smith_machine_bench_press", "smith_machine_incline_bench_press", "standing_cable_chest_fly", "standing_resistance_band_chest_fly", "band_external_shoulder_rotation", "band_internal_shoulder_rotation", "band_pull-apart", "barbell_front_raise", "barbell_rear_delt_row", "barbell_upright_row", "behind_the_neck_press", "cable_lateral_raise", "cable_rear_delt_row", "dumbbell_front_raise", "dumbbell_horizontal_internal_shoulder_rotation", "dumbbell_horizontal_external_shoulder_rotation", "dumbbell_lateral_raise", "dumbbell_rear_delt_row", "dumbbell_shoulder_press", "face_pull", "front_hold", "lying_dumbbell_external_shoulder_rotation", "lying_dumbbell_internal_shoulder_rotation", "machine_lateral_raise", "machine_shoulder_press", "monkey_row", "overhead_press", "plate_front_raise", "power_jerk", "push_press", "reverse_cable_flyes", "reverse_dumbbell_flyes", "reverse_machine_fly", "seated_dumbbell_shoulder_press", "seated_barbell_overhead_press", "seated_smith_machine_shoulder_press", "snatch_grip_behind_the_neck_press", "squat_jerk", "split_jerk", "barbell_curl", "barbell_preacher_curl", "bodyweight_curl", "cable_curl_with_bar", "cable_curl_with_rope", "concentration_curl", "dumbbell_curl", "dumbbell_preacher_curl", "hammer_curl", "incline_dumbbell_curl", "machine_bicep_curl", "spider_curl", "barbell_standing_triceps_extension", "barbell_lying_triceps_extension", "bench_dip", "close-grip_push-up", "dumbbell_lying_triceps_extension", "dumbbell_standing_triceps_extension", "overhead_cable_triceps_extension", "tricep_bodyweight_extension", "tricep_pushdown_with_bar", "tricep_pushdown_with_rope", "air_squat", "barbell_hack_squat", "barbell_lunge", "barbell_walking_lunge", "belt_squat", "body_weight_lunge", "bodyweight_leg_curl", "box_squat", "bulgarian_split_squat", "chair_squat", "dumbbell_lunge", "dumbbell_squat", "front_squat", "goblet_squat", "hack_squat_machine", "half_air_squat", "hip_adduction_machine", "jumping_lunge", "landmine_hack_squat", "landmine_squat", "leg_curl_on_ball", "leg_extension", "leg_press", "lying_leg_curl", "nordic_hamstring_eccentric", "pause_squat", "reverse_barbell_lunge", "romanian_deadlift", "safety_bar_squat", "seated_leg_curl", "shallow_body_weight_lunge", "side_lunges_(bodyweight)", "smith_machine_squat", "barbell_squat", "step_up", "zercher_squat", "assisted_chin-up", "assisted_pull-up", "back_extension", "banded_muscle-up", "barbell_row", "barbell_shrug", "block_clean", "block_snatch", "cable_close_grip_seated_row", "cable_wide_grip_seated_row", "chin-up", "clean", "clean_and_jerk", "barbell_deadlift", "deficit_deadlift", "dumbbell_deadlift", "dumbbell_row", "dumbbell_shrug", "floor_back_extension", "good_morning", "hang_clean", "hang_power_clean", "hang_power_snatch", "hang_snatch", "inverted_row", "inverted_row_with_underhand_grip", "jefferson_curl", "jumping_muscle-up", "kettlebell_swing", "lat_pulldown_with_pronated_grip", "lat_pulldown_with_supinated_grip", "muscle-up_(bar)", "muscle-up_(rings)", "one-handed_cable_row", "one-handed_lat_pulldown", "pause_deadlift", "pendlay_row", "power_clean", "power_snatch", "pull-up", "pull-up_with_a_neutral_grip", "rack_pull", "ring_pull-up", "ring_row", "seal_row", "seated_machine_row", "snatch", "snatch_grip_deadlift", "stiff-legged_deadlift", "straight_arm_lat_pulldown", "sumo_deadlift", "t-bar_row", "trap_bar_deadlift_with_high_handles", "trap_bar_deadlift_with_low_handles", "banded_side_kicks", "cable_pull_through", "clamshells", "dumbbell_romanian_deadlift", "dumbbell_frog_pumps", "fire_hydrants", "frog_pumps", "glute_bridge", "hip_abduction_against_band", "hip_abduction_machine", "hip_thrust", "hip_thrust_machine", "hip_thrust_with_band_around_knees", "lateral_walk_with_band", "machine_glute_kickbacks", "one-legged_glute_bridge", "one-legged_hip_thrust", "reverse_hyperextension", "romanian_deadlift", "single_leg_romanian_deadlift", "standing_glute_kickback_in_machine", "step_up", "ball_slams", "cable_crunch", "crunch", "dead_bug", "hanging_knee_raise", "hanging_leg_raise", "hanging_sit-up", "high_to_low_wood_chop_with_band", "horizontal_wood_chop_with_band", "kneeling_ab_wheel_roll-out", "kneeling_plank", "kneeling_side_plank", "lying_leg_raise", "lying_windshield_wiper", "lying_windshield_wiper_with_bent_knees", "machine_crunch", "mountain_climbers", "oblique_crunch", "oblique_sit-up", "plank", "plank_with_leg_lifts", "side_plank", "sit-up", "barbell_standing_calf_raise", "eccentric_heel_drop", "heel_raise", "seated_calf_raise", "standing_calf_raise", "barbell_wrist_curl", "barbell_wrist_curl_behind_the_back", "bar_hang", "dumbbell_wrist_curl", "farmers_walk", "fat_bar_deadlift", "gripper", "one-handed_bar_hang", "plate_pinch", "plate_wrist_curl", "towel_pull-up", "barbell_wrist_extension", "dumbbell_wrist_extension", "rowing_machine", "stationary_bike"]"""

In [16]:
# Call definition
def classify_exercise(user_video_uri, model_used, temp_used):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = model_used,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the exercise that the user is performing.
            It's crucial to classify the exercise accurately, choosing exclusively from the following predefined exercise indexes:
            """,
            exercises_index_list,
            "Please classify the exercise by selecting ONLY from the above list. Respond with the exact exercise index, followed by a line break."
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config={
            "max_output_tokens": 8192,
            "temperature": temp_used,
        },
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Classified exercise: {classified_exercise}\n")
    return classified_exercise

In [10]:
# Loop over 21 videos (only once per video) and process them, saving the results to a CSV file

def process_videos(videos, test_name, model_used, temp_used):
    file_path = f'{test_name}.csv'
    file_exists = os.path.isfile(file_path)
    
    for video_uri in videos:
        results = []
        start_time = time.time()  # Start time for processing the video
        try:
            classified_exercise = classify_exercise(video_uri, model_used, temp_used)  # Moved inside the loop
            end_time = time.time()  # End time for processing the video
            total_time = end_time - start_time  # Calculate total processing time for this iteration
            results.append((video_uri, classified_exercise, f"{total_time:.2f} seconds"))
        except Exception as e:
            print(f"Error processing {video_uri}: {e}")
            end_time = time.time()  # Ensure end time is captured even on error
            total_time = end_time - start_time
            results.append((video_uri, "Error", str(e), f"{total_time:.2f} seconds"))
        
        # Write results to CSV, append if file exists
        with open(file_path, 'a', newline='') as csvfile:
            fieldnames = ['input video URI', 'Call 1 (only) output', 'Total Processing Time']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if not file_exists:
                writer.writeheader()
                file_exists = True  # Ensure header is not written again
            
            for result in results:
                writer.writerow({
                    'input video URI': result[0], 
                    'Call 1 (only) output': result[1], 
                    'Total Processing Time': result[2]
                })
        if results:  # Only print if there were results processed
            print(f"Processed and saved results for {video_uri}")

In [11]:
process_videos(test_videos, "20240802-1call-pro-0.0", gemini_latest_stable, 0.0)

Classified exercise: hip_abduction_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise: bench_press_against_band

Processed and saved results for gs://gymally-mvp/exercises-videos/bench_press_against_band_ref_strengthlog.mp4
Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/bench-press-usr4-20240527-1.mp4
Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/board_bench_press_ref_strengthlog.mp4
Classified exercise: push-up_against_wall

Processed and saved results for gs://gymally-mvp/exercises-videos/bulgarian-squat-usr1-20240617-1.mp4
Classified exercise: cable_chest_press

Processed and saved results for gs://gymally-mvp/exercises-videos/cable_chest_press_ref_strengthlog.mp4
Classified exercise

In [17]:
process_videos(test_videos, "20240802-1call-flash-0.0", gemini_flash_latest_stable, 0.0)
process_videos(test_videos, "20240802-1call-flash-0.1", gemini_flash_latest_stable, 0.1)
process_videos(test_videos, "20240802-1call-flash-0.2", gemini_flash_latest_stable, 0.2)

Classified exercise: hip_abduction_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise: bench_press_against_band

Processed and saved results for gs://gymally-mvp/exercises-videos/bench_press_against_band_ref_strengthlog.mp4
Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/bench-press-usr4-20240527-1.mp4
Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/board_bench_press_ref_strengthlog.mp4
Classified exercise: dumbbell_romanian_deadlift

Processed and saved results for gs://gymally-mvp/exercises-videos/bulgarian-squat-usr1-20240617-1.mp4
Classified exercise: standing_cable_chest_fly

Processed and saved results for gs://gymally-mvp/exercises-videos/cable_chest_press_ref_strengthlog.mp4
Classi

In [18]:
# Call definition
def classify_exercise(user_video_uri, model_used, temp_used):
    vertexai.init(project="gymally-test", location="europe-west4")
    model = GenerativeModel(
        model_name = model_used,
        system_instruction = [
            """
            You're an AI Coach. Your task is to analyse a user video and identify the exercise that the user is performing.
            It's crucial to identify the strength exercise accurately, based on your knowledge on strength exercises.
            To identify the exercise, evaluate aspects like:
            - Which exercise is the user performing? E.g. Is it a deadlift, some kind of squat, some kind of pull-up... any other kind of exercise?
            - Which material is the user leveraging? E.g. a gym machine, a bench and one barbell, two barbells, resistance bands, dumbbels, kettlebells, a board, any other thing...?
            - Which group muscles does the user seem to be targeting?

            It's crucial to classify the exercise accurately, choosing exclusively from the following predefined exercise indexes:
            """,
            exercises_index_list,
            "Please classify the exercise by selecting ONLY from the above list. Respond with the exact exercise index, followed by a line break."
        ]
    )
    responses = model.generate_content(
        [
            """
            User video:
            """,
            load_video_part(user_video_uri),
            """
            Your reponse:
            """
        ],
        generation_config={
            "max_output_tokens": 8192,
            "temperature": temp_used,
        },
        safety_settings=safety_settings,
        stream=True,
    )

    classified_exercise = "".join(response.text.strip() for response in responses)
    classified_exercise = classified_exercise.strip('"')

    print(f"Classified exercise: {classified_exercise}\n")
    return classified_exercise

In [19]:
process_videos(test_videos, "20240802-1call-mid-flash-0.0", gemini_flash_latest_stable, 0.0)
process_videos(test_videos, "20240802-1call-mid-flash-0.2", gemini_flash_latest_stable, 0.2)

Classified exercise: hip_abduction_machine

Processed and saved results for gs://gymally-mvp/exercises-videos/abduction-usr1-20240527-1.mp4
Classified exercise: bar_dip

Processed and saved results for gs://gymally-mvp/exercises-videos/bar_dip_ref_strengthlog.mp4
Classified exercise: bench_press_against_band

Processed and saved results for gs://gymally-mvp/exercises-videos/bench_press_against_band_ref_strengthlog.mp4
Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/bench-press-usr4-20240527-1.mp4
Classified exercise: bench_press

Processed and saved results for gs://gymally-mvp/exercises-videos/board_bench_press_ref_strengthlog.mp4
Classified exercise: dumbbell_lunge

Processed and saved results for gs://gymally-mvp/exercises-videos/bulgarian-squat-usr1-20240617-1.mp4
Classified exercise: standing_cable_chest_fly

Processed and saved results for gs://gymally-mvp/exercises-videos/cable_chest_press_ref_strengthlog.mp4
Classified exercis